In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import json


In [ ]:
data = "../output/output.json"
chunks = []

with open(data,"r") as file:
    chunks = json.load(file)
content = [chunk["content"] for chunk in chunks]

In [ ]:
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(content,show_progress_bar=True)

embeddings_np = np.array(embeddings).astype("float32")
faiss.normalize_L2(embeddings_np)
index = faiss.IndexFlatIP(embeddings_np.shape[1])
index.add(embeddings_np)
# we make dictionary that maps index to chunk 
chunk_map = {i:chunk for i,chunk in enumerate(chunks)}

In [ ]:
query = "what is a good patent"
query_vec  = model.encode([query],show_progress_bar=True)
faiss.normalize_L2(query_vec)
k = 5
distances,indices = index.search(query_vec,k)
for idx in indices[0]:
    chunk = chunk_map[idx]
    print(f"Content: {chunk['content']}")
    print(f"chunk index: {chunk['metadata']['chunk_index']}")
    print(f"title: {chunk['metadata']['section_title']}")
    print(f"section number: {chunk['metadata']['section_number']}")
    print("*" * 80)






In [ ]:
context = "\n\n".join(
    chunk_map[idx]["content"] for idx in indices[0]
)
prompt = f"You are a AI legal assistant specializing in the patent field specifically in the United states and with the uspto. Use the following context without mentioning the context to answer the users question Context: {context} Question: {query}"


In [ ]:
import requests
import os
from dotenv import load_dotenv

load_dotenv()  # loads variables from .env
query = "what is a good patent"
query_vec  = model.encode([query],show_progress_bar=True)
faiss.normalize_L2(query_vec)
k = 5
distances,indices = index.search(query_vec,k)
# for idx in indices[0]:
#     chunk = chunk_map[idx]
#     print(f"Content: {chunk['content']}")
#     print(f"chunk index: {chunk['metadata']['chunk_index']}")
#     print(f"title: {chunk['metadata']['section_title']}")
#     print(f"section number: {chunk['metadata']['section_number']}")
#     print("*" * 80)


context = "\n\n".join(
    chunk_map[idx]["content"] for idx in indices[0]
)
prompt = f"""You are an AI legal assistant with expert knowledge of United States patent law and the USPTO (United States Patent and Trademark Office).  
Use the following context **internally** to answer the user's question, but **do not mention the context, its existence, or that you used it** in your response.  
Respond as if the knowledge is your own and part of your general expertise.  
If the context does not fully answer the question, use your own legal knowledge to fill in any gaps as needed. Do not show your thinking answer directly

Context: {context}  
Question: {query}"""



from together import Together

client = Together() # auth defaults to os.environ.get("TOGETHER_API_KEY")

response = client.chat.completions.create(
    model="Qwen/Qwen3-235B-A22B-Thinking-2507",
    messages=[
      {
        "role": "user",
        "content": prompt
                    }
    ]
)
print(response.choices[0].message.content)

